<a href="https://colab.research.google.com/github/cathay/cnn/blob/master/cat_and_dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

#Import Google drive functions
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
#Utilities to upload files
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())


In [0]:
#Authorize Google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#download train and test images
download = drive.CreateFile({'id': '1EVPUykY4KwHvWL-umtLhJIfGhCFtHqn8'})
download.GetContentFile('train.zip')
download = drive.CreateFile({'id': '1l4vKliu1LB5Y65ZMvsSDrJiPG2mKvdqN'})
download.GetContentFile('tests.zip')

#Extract files
zip_ref = zipfile.ZipFile('train.zip', 'r')
zip_ref.extractall('tmp')
zip_ref = zipfile.ZipFile('tests.zip', 'r')
zip_ref.extractall('tmp')
zip_ref.close()

In [0]:
def get_im_cv2(path, w, h):
  img = cv2.imread(path)
  resized = cv2.resize(img, (w,h), cv2.INTER_LINEAR) 
  return resized

In [31]:
TRAININNG_DIRECTORY='tmp/train'
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

df = pd.DataFrame({'file': os.listdir(TRAININNG_DIRECTORY)})

df['animal'] = df['file'].map(lambda file_name: file_name.split('.')[0])
df['Y'] = df['animal'].map(lambda x: np.array([1,0]).transpose() if x !='cat' else np.array([0,1]).transpose())
df['X'] = df['file'].map(lambda file: get_im_cv2(TRAININNG_DIRECTORY + "/" + file, IMAGE_WIDTH, IMAGE_HEIGHT))

25000
12500


,file,animal,Y,X
0,cat.5721.jpg,cat,"[0, 1]","[[[152, 174, 209], [152, 174, 209], [152, 174,..."
1,cat.12084.jpg,cat,"[0, 1]","[[[96, 93, 94], [138, 117, 99], [144, 131, 126..."
2,dog.11124.jpg,dog,"[1, 0]","[[[27, 49, 45], [51, 77, 73], [22, 47, 43], [4..."
3,dog.5092.jpg,dog,"[1, 0]","[[[158, 180, 197], [154, 176, 193], [153, 174,..."
4,cat.7605.jpg,cat,"[0, 1]","[[[101, 152, 185], [100, 151, 184], [104, 155,..."


In [0]:
df.head()